# Assignment 4: Prep work for midterm
### Writing functions for more efficient coding / Group: Coast Live Oaks

First I'm going to import all the libraries I'll need in this notebook. 

In [20]:
# for wrangling data and geospatial data
import pandas as pd
import geopandas as gpd

Then I'm going to upload my census data on housing characteristics in LA County. 

In [21]:
# read csv of ACS data on housing characteristics
df = pd.read_csv('data/housingData-ACS2022.csv')

I'm going to briefly explore the dataset. 

In [22]:
df.FIPS.head()

0    6037101110
1    6037101122
2    6037101220
3    6037101221
4    6037101222
Name: FIPS, dtype: int64

I've run into the leading-zero problem, so I'm going to define a function that adds the zero and converts the FIPS code to a string. 

In [23]:
# with arguments, function can be applied to other columns
def add_leading_zero(column, items):
    df[column] = df[column].astype(str).str.zfill(items) # convert column to string and use zfill to add leading zero

add_leading_zero('FIPS', 11)

In [24]:
# check that function worked
df.FIPS.head()

0    06037101110
1    06037101122
2    06037101220
3    06037101221
4    06037101222
Name: FIPS, dtype: object

The function worked: the FIPS items were converted from integers to objects. 

Now I want to check for columns with all null values. 

In [25]:
df.columns[df.isna().all()].tolist()

[]

In [26]:
df.head()

,FIPS,NAME,DP04_0001E,DP04_0001M,DP04_0002E,DP04_0002M,DP04_0003E,DP04_0003M,DP04_0004E,DP04_0004M,...,DP04_0139PE,DP04_0139PM,DP04_0140PE,DP04_0140PM,DP04_0141PE,DP04_0141PM,DP04_0142PE,DP04_0142PM,DP04_0143PE,DP04_0143PM
0,06037101110,Census Tract 1011.10; Los Angeles County; Cali...,1652,97,1551,130,101,73,0,4.5,...,16.2,12,3.2,3.4,9.2,6.3,49.6,12.2,(X),(X)
1,06037101122,Census Tract 1011.22; Los Angeles County; Cali...,1427,170,1383,160,44,47,0,3.7,...,2.8,4.5,11.6,16.4,3.2,6.1,82.4,18.3,(X),(X)
2,06037101220,Census Tract 1012.20; Los Angeles County; Cali...,1372,207,1349,208,23,21,0,6.6,...,5.5,5,21.6,15,13,8,41.6,12.3,(X),(X)
3,06037101221,Census Tract 1012.21; Los Angeles County; Cali...,1549,285,1424,293,125,75,0,10.9,...,3.4,4.2,16.2,10.7,6.5,6.7,64.1,17.8,(X),(X)
4,06037101222,Census Tract 1012.22; Los Angeles County; Cali...,1006,138,928,146,78,64,0,59.1,...,3.2,3.7,6.4,5.8,0.8,2.2,59,13,(X),(X)


There aren't any columns with all null values, so there's nothing for me to remove. Now I'm going to create two subsetted dataframes that contains my variables of interest. The first will be for when houses were built, the second will be for houses' source of heating fuel. 

In [27]:
# create list
columns_vintage = [
                        'FIPS',
                        'DP04_0017E',
                    	'DP04_0018E',
                        'DP04_0019E',
                        'DP04_0020E',
                    	'DP04_0021E',
                    	'DP04_0022E',
                        'DP04_0023E',
                        'DP04_0024E',
                        'DP04_0025E',
                        'DP04_0026E'
]

# define subset
df_vintage = df[columns_vintage]

In [28]:
list(df_vintage)

['FIPS',
 'DP04_0017E',
 'DP04_0018E',
 'DP04_0019E',
 'DP04_0020E',
 'DP04_0021E',
 'DP04_0022E',
 'DP04_0023E',
 'DP04_0024E',
 'DP04_0025E',
 'DP04_0026E']

In [29]:
columns = list(df_vintage)

In [30]:
df_vintage.columns = [
                     'FIPS',
                     '2020 or Later',
                     '2010 to 2019',
                     '2000 to 2009',
                     '1990 to 1999',
                     '1980 to 1989',
                     '1970 to 1979',
                     '1960 to 1969',
                     '1950 to 1959',
                     '1940 to 1949',
                     '1939 or Earlier'
]

df_vintage.head()

,FIPS,2020 or Later,2010 to 2019,2000 to 2009,1990 to 1999,1980 to 1989,1970 to 1979,1960 to 1969,1950 to 1959,1940 to 1949,1939 or Earlier
0,06037101110,0,39,48,87,79,119,213,516,223,328
1,06037101122,0,5,247,35,68,176,469,286,31,110
2,06037101220,9,32,58,81,72,231,187,232,306,164
3,06037101221,0,35,113,353,191,300,221,83,78,175
4,06037101222,0,35,0,11,445,138,98,162,45,72


Now I'm going to subset the data again, for heating fuels. But to make things faster I'd like to create a subsetting function. 

In [31]:
# the function has arguments for the df, the columns in the subset, the subsetted df, and any number of column names
def subset_census(df_original, columns_list, df_subset, *column_names):
    
    columns_list.extend(column_names) # the list of columns in the subset can be extended by any number

    df_subset = df_original[columns_list] # define the subsetted df

    return df_subset

Now I'm going to test the function. 

In [32]:
# define variables
columns_test = ['DP04_0026E', 'DP04_0025E', 'DP04_0024E', 'DP04_0023E']
df_test = df[columns_test]

# call function
subset_census(df, columns_test, df_test,'DP04_0026E', 'DP04_0025E', 'DP04_0024E', 'DP04_0023E')

# see if it worked
df_test.head()

,DP04_0026E,DP04_0025E,DP04_0024E,DP04_0023E
0,328,223,516,213
1,110,31,286,469
2,164,306,232,187
3,175,78,83,221
4,72,45,162,98


It did work. Now I'm going to use the function for my other variable of interest: fuel type for houses. 

In [35]:
# define columns of interest
fuel_columns = ['DP04_0063E',
                'DP04_0064E',
                'DP04_0065E',
                'DP04_0066E',
                'DP04_0067E',
                'DP04_0068E',
                'DP04_0069E',
                'DP04_0070E',
                'DP04_0071E']

# define dataframe to be subsetted
df_fuel = df[fuel_columns]

# call subset function
subset_census(df, fuel_columns, df_fuel, 
                'DP04_0063E',
                'DP04_0064E',
                'DP04_0065E',
                'DP04_0066E',
                'DP04_0067E',
                'DP04_0068E',
                'DP04_0069E',
                'DP04_0070E',
                'DP04_0071E')

df_fuel.head()

,DP04_0063E,DP04_0064E,DP04_0065E,DP04_0066E,DP04_0067E,DP04_0068E,DP04_0069E,DP04_0070E,DP04_0071E
0,1070,35,432,0,0,0,0,0,14
1,1211,14,135,0,0,23,0,0,0
2,959,48,308,0,0,0,0,0,34
3,891,57,448,0,0,0,0,0,28
4,361,37,490,12,0,0,7,0,21


Now I need to change the column names. 

In [36]:
columns = list(df_fuel)

In [37]:
df_fuel.columns = [ 'Utlity gas',
                    'Bottled, tank, or LP gas',
                    'Electricity',
                    'Fuel oil, kersone, etc.',
                    'Coal or coke',
                    'Wood',
                    'Solar energy',
                    'Other',
                    'No fuel used']

df_fuel.head()

,Utlity gas,"Bottled, tank, or LP gas",Electricity,"Fuel oil, kersone, etc.",Coal or coke,Wood,Solar energy,Other,No fuel used
0,1070,35,432,0,0,0,0,0,14
1,1211,14,135,0,0,23,0,0,0
2,959,48,308,0,0,0,0,0,34
3,891,57,448,0,0,0,0,0,28
4,361,37,490,12,0,0,7,0,21


OK: on reflection, it seems like my `subset_census()`function actually creates more work, because now I have to define one of the columns variable. This means I have to paste in those codes twice. This is exactly the type of work I was trying to avoid...I'm going to have to reconsider that for the midterm, but for now I'm tapped out...